# Crypto-Assets Cointegration Project

### Modules

In [1]:
import binance_api
import numpy as np
import pandas as pd

import statsmodels
from statsmodels.tsa.stattools import coint
import statsmodels.api as stat
import statsmodels.tsa.stattools as ts

import matplotlib.pyplot as plt
from datetime import datetime, date

# https://www.marketcalls.in/amibroker/computing-cointegration-and-augmented-dickey-fuller-test-in-amibroker-using-python.html#:~:text=Augmented%20Dicky%20Fuller%20test%20is,stationary%20and%20cointegrated%20or%20not.&text=The%20Augmented%20Dicky%20Fuller%20test,want%20to%20reject%20this%20hypothesis.


### Default Values

In [9]:
# default values
nbr_of_coins = 20
interval = '1d'
lookback_period = 40
startTime = round(datetime(2020, 1, 1).timestamp()) * 1000
endTime = round(datetime(2022, 1, 1).timestamp()) * 1000

### Get coins with most volume on Binance

In [3]:
# Get all coins, sort them by volume and keep the top x nbr of coins
tickers = pd.DataFrame(binance_api.get_all_tickers())
tickers['volume_usd'] = tickers.apply(lambda row: float(row['volume']) * float(row['lastPrice']), axis=1)
tickers = tickers.sort_values(by='volume_usd', ascending=False)
top_tickers = tickers[:nbr_of_coins]
print(top_tickers['symbol'])

88          BTCUSDT
61          ETHUSDT
138    1000SHIBUSDT
15          XRPUSDT
46         DOGEUSDT
97         GALAUSDT
34          BTCBUSD
64         LUNAUSDT
86          SOLUSDT
92         SANDUSDT
24          ADAUSDT
25         AVAXUSDT
111         ETCUSDT
99         MANAUSDT
65       PEOPLEUSDT
79          FTMUSDT
82          BNBUSDT
112         ETHBUSD
16        MATICUSDT
140         DOTUSDT
Name: symbol, dtype: object


### Historical Data

In [4]:
historical_data = {}
for i in range(0, len(top_tickers)):
    historical_data[top_tickers.iloc[i]['symbol']] = pd.DataFrame(binance_api.get_historical_prices(top_tickers.iloc[i]['symbol'], interval, startTime, endTime))


### Cointegration Example

In [10]:
data_1 = np.asarray(historical_data['BTCUSDT'][4].tail(lookback_period)).astype(float)
data_2 = np.asarray(historical_data['ETHUSDT'][4].tail(lookback_period)).astype(float)
result = stat.OLS(data_1, data_2).fit()
a = ts.adfuller(result.resid)
print('p-value: {}'.format(a[1]))

p-value: 0.06873258229192404


### Cointegration Matrix

In [35]:
symbols = list(historical_data.keys())
coint_matrix = pd.DataFrame(index=symbols, columns=symbols)

for i in range(0, len(coint_matrix)):
    for j in range(0, len(coint_matrix)):
        if i != j:
            data_1 = np.asarray(historical_data[coint_matrix.index[i]][4].tail(lookback_period)).astype(float)
            data_2 = np.asarray(historical_data[coint_matrix.index[j]][4].tail(lookback_period)).astype(float)
            if len(data_1) == len(data_2):
                result = stat.OLS(data_1, data_2).fit()
                a = ts.adfuller(result.resid)
                pvalue = a[1]
                coint_matrix[coint_matrix.index[i]][coint_matrix.index[j]] = pvalue

# Color cells accordingly
coint_matrix.style.applymap(lambda x: "background-color: red" if x>0.1 else "background-color: green")
coint_matrix

,BTCUSDT,ETHUSDT,1000SHIBUSDT,XRPUSDT,DOGEUSDT,GALAUSDT,BTCBUSD,LUNAUSDT,SOLUSDT,SANDUSDT,ADAUSDT,AVAXUSDT,ETCUSDT,MANAUSDT,PEOPLEUSDT,FTMUSDT,BNBUSDT,ETHBUSD,MATICUSDT,DOTUSDT
BTCUSDT,NaN,0.068951,0.02267,0.212514,0.001805,0.027732,0.032634,0.636931,0.015949,0.614056,0.144988,0.614091,0.202258,0.246677,NaN,0.607545,0.044474,0.068988,0.640339,0.099263
ETHUSDT,0.068733,NaN,0.046602,0.186068,0.073209,0.000333,0.069657,0.785983,0.009713,0.125238,0.096402,0.73181,0.205304,0.417646,NaN,0.589136,0.161572,0.094291,0.762612,0.057581
1000SHIBUSDT,0.022362,0.045655,NaN,0.337795,0.028345,0.040084,0.022227,0.624578,0.054379,0.401083,0.000073,0.656146,0.180743,0.000007,NaN,0.987544,0.168571,0.045214,0.226153,0.016638
XRPUSDT,0.213615,0.186714,0.338506,NaN,0.128448,0.001273,0.213766,0.416561,0.079138,0.000771,0.162882,0.467554,0.76978,0.201131,NaN,0.76403,0.284736,0.188097,0.394649,0.313419
DOGEUSDT,0.001783,0.073355,0.028654,0.128376,NaN,0.000004,0.001841,0.458444,0.072506,0.318225,0.005795,0.530607,0.508031,0.013702,NaN,0.964887,0.019347,0.073218,0.263398,0.079117
GALAUSDT,0.026064,0.000302,0.036619,0.001134,0.000003,NaN,0.026328,0.283276,0.002254,0.140953,0.000012,0.06026,0.000005,0.000122,NaN,0.672363,0.004906,0.000295,0.600799,0.013499
BTCBUSD,0.032627,0.069874,0.022534,0.212658,0.001864,0.028001,NaN,0.637747,0.016184,0.614869,0.144023,0.61376,0.199684,0.244498,NaN,0.602574,0.044281,0.069894,0.641025,0.098809
LUNAUSDT,0.630333,0.79023,0.616778,0.392791,0.434768,0.848738,0.631206,NaN,0.468661,0.014114,0.334011,0.222856,0.573436,0.828017,NaN,0.166179,0.779043,0.791122,0.107935,0.25081
SOLUSDT,0.015793,0.009786,0.05556,0.078866,0.072448,0.002445,0.016025,0.489663,NaN,0.453902,0.021299,0.671371,0.523848,0.15586,NaN,0.986698,0.034674,0.009196,0.396261,0.062261
SANDUSDT,0.615357,0.124228,0.408407,0.000701,0.313377,0.138674,0.616202,0.023737,0.453067,NaN,0.0,0.010259,0.769729,0.904457,NaN,0.966436,0.618923,0.125025,0.556707,0.935588
